A notebook to process results saved as pandas data frames (i.e., combined Tensorboard logs).

In [28]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import autograd.numpy as np
import scipy.stats as stats
import torch
import tensorflow as tf

In [30]:
import cadgan
import cadgan.glo as glo
import cadgan.imutil as imutil
import cadgan.ex.df as dfutil
import cadgan.util as util
import IPython

## Load dataframe file

In [34]:
# df = pd.read_pickle(glo.share_path('dataframe_results','face_n1_vgglayers_lr'))
# fname = 'face_n1_vgglayers_lr'
# fname = 'bedroom_all.pkl'
fname = 'bedroom_kernels.pkl'
# fname = 'bridge_all.pkl'
# fname = 'cub_all.pkl'
# fname = 'celeba_all.pkl'
# fname = 'tower_all.pkl'
df_fpath = glo.result_folder('dataframe_results', fname)
df = pd.read_pickle(df_fpath)

In [35]:
def write_text(fname, content):
    with open(fname, 'w') as f:
        f.write(content)

## Display images in a dataframe

In [36]:
df.columns.values

array(['cond_feat', 'cond_imgs', 'cond_path', 'depth_process', 'device',
       'extractor_layers', 'extractor_type', 'feat_imgs', 'g_max',
       'g_min', 'g_path', 'g_type', 'img_log_steps', 'img_size',
       'ini_imgs', 'iteration', 'kernel', 'kparams', 'log_path', 'logdir',
       'lr', 'n_init_resample', 'n_opt_iter', 'n_sample', 'out_imgs',
       'prior_ini', 'seed', 'texture', 'w_input', 'zpen'], dtype=object)

List unique values for relevant columns

In [37]:
uniq_cols = ['cond_path', 'extractor_layers', 'extractor_type', 'kernel', 
             'kparams', 'lr', 'n_init_resample','n_sample', 'zpen']
for c in uniq_cols:
    print('# {}'.format(c))
    print(df[c].unique())
    print()

# cond_path
['/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_6.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_2.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_5.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_3.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n1_3.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_8.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_10.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_7.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n1_2.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_1.txt'
 '/notebooks/psangkloy3/gdrive/condgan_share/sample_list/bedroom_list/files_n2_9.txt'
 '/notebooks/psangkloy3/gdrive/condgan_sh

In [38]:
#filter parameters to show
# df = df[['cond_imgs','out_imgs','lr','extractor_layers','iteration','img_size','extractor_type','cond_path']] 
df_sliced = df[['cond_path', 'cond_imgs','ini_imgs', 'out_imgs', 
                'n_init_resample',
                'n_sample',
                'kernel', 
                'kparams', 
                  'lr',
                'extractor_type',
                'extractor_layers', 
                'log_path',
#                 'zpen'
               ]] 
df_sliced = df_sliced.sort_values(by=['cond_path', 'n_sample', 'kernel'])
df_sliced = df_sliced.drop(columns=['cond_path', 'n_sample'])

In [39]:
images_formatter = lambda imgs: dfutil.images_formatter(imgs, col=2, width=96)
html = dfutil.df_to_html(df_sliced, img_formatter=images_formatter)
IPython.display.HTML(html) 

write_fname = '{}-all.html'.format(fname)
print('writing to {}'.format(write_fname))
write_text(fname=write_fname, content=html)

writing to bedroom_kernels.pkl-all.html


In [40]:
def filter_save(df, filters):
    df_filtered = df
    
    for fil in filters:
        col, val = fil
        df_filtered = df_filtered.loc[ (df_filtered[col]==val) ]
    filters_summary ='_'.join(['{}{}'.format(c, v) for (c,v) in filters])
    # df_filtered = df_sliced.loc[ ( df_sliced['lr']=="0.05") ]

    html = dfutil.df_to_html(df_filtered)
    write_text(fname='{}-{}.html'.format(fname, filters_summary), content=html)
# IPython.display.HTML(html)

In [41]:
filters = [('lr', '0.05')]
filter_save(df_sliced, filters)

In [42]:
filters = [('lr', '0.05'), ('kernel', 'linear')]
filter_save(df_sliced, filters)

In [43]:
filters = [('lr', '0.05'), ('kernel', 'imq')]
filter_save(df_sliced, filters)

--------

In [15]:
#do some filtering based on parameters, you can google dataframe querying for a more advance usage
df_final = df

#filter from parameter value, can also use &
df_final = df_final.loc[df_final['cond_path']=='/notebooks/psangkloy3/gdrive/condgan_share/face_train_list/files_n1_3.txt']
df_final = df_final.loc[df_final['lr']=='0.05']
df_final = df_final.loc[(df_final['extractor_layers']=="['35']") | (df_final['extractor_layers']=="['17']" )]


In [16]:
#save results to html file
with open('bird_hed.html','w+') as f:
    f.write(html)